In [8]:
# read list of nuclear charges, give energy predictions up to nth order

In [74]:
import numpy as np
import pandas as pd
import qml
import scipy.spatial as scs
import scipy.interpolate as sci

In [ ]:
def get_predictions(comb = [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 7]):
    #rho, dsingle, dneigh1, dneigh2, dneigh3, dneigh4, dneigh5 = read_densities()
        
    E =0 #-758.072029908548 # base energy
    deltaZ = np.array(comb) - 6
        
    zs = np.array([int(_) for _ in comb])
    zsref = np.zeros(20) + 6
    
    E -= get_nucnuc(zsref)
    E += get_nucnuc(zs)
    
    # 0-th order, no rotation necessary, should be hard zero
    #ds = np.linalg.norm(grid_points - c.coordinates[0], axis=1)
    #es = (rho * grid_weights / ds).sum()
    #for idx, Z in enumerate(comb):
    #    if deltaZ[idx] == 0:
    #        continue
    #    E += np.sum(deltaZ[idx] * rho * grid_weights / ds)
    
    # 1st order
    changed_site, grid_points, grid_densweight = read_grid_first_order()
    for idx, Z in enumerate(comb):
        if deltaZ[idx] == 0:
            continue
        mapping = build_reindexing_1(idx, changed_site)
        
        for j in range(20): 
            if deltaZ[mapping[j]] == 0:
                continue
            ds = np.linalg.norm(grid_points - c.coordinates[j], axis=1)*1.8897259885789
            E += np.sum(deltaZ[idx] * deltaZ[mapping[j]] * grid_densweight / ds)/2
    
    # 2nd order
    for idx_i, Z_i in enumerate(comb):
        if deltaZ[idx_i] == 0:
            continue
        for idx_j, Z_j in enumerate(comb):
            if deltaZ[idx_j] == 0:
                continue
            
            # t_i: target for idx_i after rotation
            # deriv_pair: part of derivative after pair-mapping
            # deriv_single: part of derivative after single-mapping
            t_i, t_j, deriv_pair, deriv_single = get_deriv(idx_i, idx_j)
            
            # pairwise mapping
            if idx_i != idx_j:
                try:
                    mapping = build_reindexing_2_merged(idx_i, idx_j, t_i, t_j)
                except:
                    print (idx_i, idx_j, t_i, t_j)
                for j in range(20): 
                    if deltaZ[mapping[j]] == 0:
                        continue
                    ds = np.linalg.norm(grid_points - c.coordinates[j], axis=1)
                    E += np.sum(deltaZ[idx] * deltaZ[mapping[j]] * grid_densweight / ds) / 6
            
            # single mapping
            mapping = build_reindexing_1(idx_j, 0)
            for j in range(20): 
                if deltaZ[mapping[j]] == 0:
                    continue
                ds = np.linalg.norm(grid_points - c.coordinates[j], axis=1)
                E += np.sum(deltaZ[idx] * deltaZ[mapping[j]] * grid_densweight / ds) / 6
            
    return E
    # 0-1 neigh1
    # 0-2 neigh2
    # 0-8 neigh3
    # 0-10 neigh4
    # 0-16 neigh5
get_predictions([int(_) for _ in '57766576666555776675'])

/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/guido/miniconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide


1 15 0 8
6 11 0 16
11 6 0 16
12 0 0 8


In [4]:
c = qml.Compound('../../test/c20.xyz')

In [84]:
def _get_nn(refsite):
    distances = np.linalg.norm(c.coordinates - c.coordinates[refsite], axis=1)
    return np.argsort(distances)[1:4]
def build_reindexing_1(refsite, ontosite):
    cog = np.mean(c.coordinates, axis=0)
    valid = False
    for Ann in _get_nn(refsite):
        for Bnn in _get_nn(ontosite):
            A = c.coordinates[[refsite, Ann]]
            B = c.coordinates[[ontosite, Bnn]]
            rot = scs.transform.Rotation.match_vectors(A, B)[0]
            transformed = rot.apply(c.coordinates)
            found = []
            for site in range(len(c.coordinates)):
                ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
                if min(ds) < 1e-5:
                    found.append(np.argmin(ds))
            if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite] == ontosite:
                valid = True
                break
        if valid:
            break
    if not valid:
        raise ValueError('no solution')
    return found

build_reindexing_1(5, 0)

[4, 3, 11, 12, 13, 0, 1, 2, 9, 10, 16, 17, 18, 14, 5, 8, 15, 19, 6, 7]

In [209]:
def build_reindexing_2_inverse(refsite1, refsite2, ontosite1, ontosite2):
    valid = False
    coordinates = np.copy(c.coordinates)*(-1)
    A = c.coordinates[[refsite1, refsite2]]
    B = coordinates[[ontosite1, ontosite2]]
    # rotate one first
    a = np.cross(A[0], (0,0,1))
    b = np.cross(B[0], (0, 0, 1))
    rot = scs.transform.Rotation.match_vectors([A[0], a], [B[0], b])[0]
    transformed = rot.apply(c.coordinates)
    #print ('step1', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))

    # rotate second
    A = c.coordinates[[refsite1, refsite2]]
    B = transformed[[ontosite1, ontosite2]]
    rot = scs.transform.Rotation.match_vectors(A, B)[0]
    transformed2 = rot.apply(transformed)
    #print ('step2', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed2[[ontosite1, ontosite2]] , axis=1))
    transformed = transformed2
    if max(np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1)) > 1e-5:
        raise ValueError('no rotation')
    found = []
    #for site in range(20):
    #    if site in (refsite1, refsite2, ontosite1, ontosite2):
    #        print ('B', *c.coordinates[site])
    #        print ('N', *transformed[site])
    #    else:
    #        print ('C', *c.coordinates[site])
    #        print ('C', *transformed[site])
    for site in range(len(coordinates)):
        ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
        if min(ds) < 1e-5:
            found.append(np.argmin(ds))
    #print (set(found))
    if set(found) == set([_ for _ in range(len(coordinates))]) and found[refsite1] == ontosite1 and found[refsite2] == ontosite2:
        pass
    else:
        raise ValueError('no solution')
    return found
def build_reindexing_2(refsite1, refsite2, ontosite1, ontosite2):
    valid = False
    
    A = c.coordinates[[refsite1, refsite2]]
    B = c.coordinates[[ontosite1, ontosite2]]

    # rotate one first
    a = np.cross(A[0], (0,0,1))
    b = np.cross(B[0], (0, 0, 1))
    rot = scs.transform.Rotation.match_vectors([A[0], a], [B[0], b])[0]
    transformed = rot.apply(c.coordinates)
    #print ('step1', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))

    # rotate second
    A = c.coordinates[[refsite1, refsite2]]
    B = transformed[[ontosite1, ontosite2]]
    rot = scs.transform.Rotation.match_vectors(A, B)[0]
    transformed2 = rot.apply(transformed)
    #print ('step2', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed2[[ontosite1, ontosite2]] , axis=1))
    transformed = transformed2
    if max(np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1)) > 1e-5:
        raise ValueError('no rotation')

    found = []
    for site in range(len(c.coordinates)):
        ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
        if min(ds) < 1e-5:
            found.append(np.argmin(ds))
    if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite1] == ontosite1 and found[refsite2] == ontosite2:
        pass
    else:
        raise ValueError('no solution')
    return found
def build_reindexing_2_both(a, b, c, d):
    try:
        return build_reindexing_2(a, b, c, d)
    except ValueError:
        pass
    return build_reindexing_2_inverse(a, b, c, d)
#build_reindexing_2(1, 5, 0, 2)
build_reindexing_2_merged(0, 6, 0, 2)

ValueError: no rotation

In [28]:
def read_DENSITY(fn):
    with open(fn, 'r') as fh:
        _ = np.fromfile(fh, 'i4')
        q = _[3:-1].view(np.float64)
        ccdensity = q.reshape((-1, 10))
    ccdensity = ccdensity[:, 1:6]
    return ccdensity[:, :3], ccdensity[:, 3], ccdensity[:, 4]

def read_grid_first_order():
    changed_site = 0
    delta = 0.1
    
    upgrid, upweight, updens = read_DENSITY('c20-data/derivatives/order-1/site-0-up/DENSITY')
    dngrid, dnweight, dndens = read_DENSITY('c20-data/derivatives/order-1/site-0-dn/DENSITY')
    
    if not np.allclose(upgrid, dngrid):
        raise ValueError('Grid?')
        
    if not np.allclose(upweight, dnweight):
        raise ValueError('Grid?')
    
    return changed_site, upgrid, ((updens - dndens) / delta)*upweight

(0, array([[-14.45979167,  -7.67919738,  -4.7844303 ],
        [-14.12594505,  -6.47848004,  -4.16214482],
        [-13.98485046,  -7.04498829,  -6.58628751],
        ...,
        [ 14.12594505,   6.47848004,   4.16214293],
        [ 14.45978978,   7.67920116,   4.78443219],
        [ 15.35325833,   6.6146531 ,   3.73665988]]), array([7.80818106e-25, 6.67525811e-23, 2.61010655e-25, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))

In [58]:
def get_nucnuc(zs):
    ds = scs.distance.squareform(scs.distance.pdist(c.coordinates))*1.8897259885789
    q = np.outer(zs, zs)/ds
    np.fill_diagonal(q, 0)
    return q.sum()/2    

In [81]:
def rotate_density(origgrid, origdens, target):
    rot = get_rotation_1(target, 0)
    newgrid = rot.apply(origgrid)
    transform = sci.Rbf(newgrid[:, 0], newgrid[:, 1], newgrid[:, 2], origdens, function='thin_plate')
    return transform(origgrid[:, 0], origgrid[:, 1], origgrid[:, 2])
    
def get_deriv(i, j):
    """ Returns 
    t_i : atom index of i after rotation
    t_j : atom index of j after rotation
    deriv_pair : the density to be integrated after pairwise rotation
    deriv_single : density to be integrated after single rotation """
    
    d = np.linalg.norm(c.coordinates[i] - c.coordinates[j])
    geo = np.argmin(np.abs(np.array(sorted(set(np.round(scs.distance.pdist(c.coordinates), 2)))) - d))
    sites = (0, (1, 2, 8, 10, 16)[geo])
    i, j = sites
    delta = 0.05
    
    assert i == 0
    midgrid, midweight, middens = read_DENSITY('c20-data/derivatives/order-0/site-all-cc/DENSITY')
    # prefill output
    deriv_single = np.zeros(middens.shape)
    deriv_pair = np.zeros(middens.shape)
    
    iupgrid, iupweight, iupdens = read_DENSITY('c20-data/derivatives/order-1/site-0-up/DENSITY')
    idngrid, idnweight, idndens = read_DENSITY('c20-data/derivatives/order-1/site-0-dn/DENSITY')
    
    if i == j:
        deriv_single = (iupdens + idndens - 2 * middens)/(delta**2)
    else:
        rhojup = iupdens
        rhojdn = idndens
        upgrid, upweight, updens = read_DENSITY('c20-data/derivatives/order-2/site-0-%d-up/DENSITY' % j)
        dngrid, dnweight, dndens = read_DENSITY('c20-data/derivatives/order-2/site-0-%d-dn/DENSITY' % j)
        
        deriv_pair = (updens + dndens + 2 * middens - iupdens - idndens) / (2 * delta**2)
        deriv_single = (- rhojup - rhojdn) / (2 * delta**2)
    
    return i, j, deriv_pair, deriv_single
get_deriv(0, 1)

(0, 1, array([-2.99319507e-24, -1.92319235e-20, -4.45285633e-24, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), array([-1.40363029e-23, -7.45965882e-20, -1.46127557e-23, ...,
        -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]))

In [103]:
np.linalg.norm(c.coordinates[0] - c.coordinates[1])

1.4941261703959974

In [216]:
# 1, 5, 0, 2
refsite1, refsite2, ontosite1, ontosite2 = 0,6,0,2
#coordinates = np.copy(c.coordinates)*(-1)
A = c.coordinates[[refsite1, refsite2]]
B = c.coordinates[[ontosite1, ontosite2]]

# rotate one first
a = np.cross(A[1], (0,0,1))
b = np.cross(B[1], (0, 0, 1))
rot = scs.transform.Rotation.match_vectors([A[1], a], [B[1], b])[0]
transformed = rot.apply(c.coordinates)
print ('step1', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))

# rotate second
A = c.coordinates[[refsite1, refsite2]]
B = transformed[[ontosite1, ontosite2]]
print (A, B)
rot = scs.transform.Rotation.match_vectors(A, B)[0]
print (rot.as_euler('zyx', degrees=True))
transformed2 = rot.apply(transformed)
print ('step2', np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed2[[ontosite1, ontosite2]] , axis=1))



step1 [3.75594755 3.01252493]
[[-0.74706308 -1.02824412  1.66373333]
 [-1.9558366  -0.63548844 -0.39275372]] [[-0.6715409  -0.1181836  -1.97951087]
 [-0.40292391  1.92557024 -0.71640031]]
[  57.1037555    14.32654329 -129.55207668]
step2 [4.67027847e-07 4.67101023e-07]


In [180]:
#  c' = (-a*a - b*b)/c
c.coordinates

array([[-7.47063080e-01, -1.02824412e+00,  1.66373333e+00],
       [-1.20877346e+00,  3.92754310e-01,  1.66373333e+00],
       [-0.00000000e+00,  1.27097963e+00,  1.66373333e+00],
       [ 1.20877346e+00,  3.92754310e-01,  1.66373333e+00],
       [ 7.47063080e-01, -1.02824412e+00,  1.66373333e+00],
       [-1.20877187e+00, -1.66373428e+00,  3.92755160e-01],
       [-1.95583660e+00, -6.35488440e-01, -3.92753720e-01],
       [-1.95583588e+00,  6.35490290e-01,  3.92754300e-01],
       [-1.20877295e+00,  1.66373389e+00, -3.92753530e-01],
       [-1.30000000e-07,  2.05648767e+00,  3.92754760e-01],
       [ 1.20877402e+00,  1.66373326e+00, -3.92752910e-01],
       [ 1.95583604e+00,  6.35489520e-01,  3.92754760e-01],
       [ 1.95583634e+00, -6.35489350e-01, -3.92753530e-01],
       [ 1.20877248e+00, -1.66373345e+00,  3.92756830e-01],
       [ 9.80000000e-07, -2.05648829e+00, -3.92751510e-01],
       [-7.47061760e-01,  1.02824567e+00, -1.66373297e+00],
       [ 7.47063080e-01,  1.02824565e+00

In [319]:
def build_reindexing_2_merged(refsite1, refsite2, ontosite1, ontosite2):
    for inverse in (True, False):
        for asc in (True, False):
            for mirror in (True, False):
                for mirrorafter in (True, False):
                    try:
                        return do_it(refsite1, refsite2, ontosite1, ontosite2, inverse, asc, mirror, mirrorafter)
                    except ValueError:
                        continue
    raise ValueError('No luck.')
def do_it(refsite1, refsite2, ontosite1, ontosite2, inverse, asc, mirror, mirrorafter):
    #print (inverse, asc, mirror)
    valid = False
    if inverse:
        coordinates = np.copy(c.coordinates)*(-1)
    else:
        coordinates = np.copy(c.coordinates)
    
    A = c.coordinates[[refsite1, refsite2]]
    B = coordinates[[ontosite1, ontosite2]]
    
    if mirror:
        ax1 = A.sum(axis=0)
        ax2 = B.sum(axis=0)
        a = ax1 - ax2
        for site in range(20):
            v = coordinates[site].copy()
            coordinates[site] = v- 2*a*np.dot(v, a) / np.dot(a, a)
        transformed = coordinates
    else:
        A = c.coordinates[[refsite1, refsite2]]
        B = coordinates[[ontosite1, ontosite2]]
        #print (np.linalg.norm(c.coordinates[[refsite1, refsite2]] - coordinates[[ontosite1, ontosite2]] , axis=1))
        if asc:
            index = 0
        else:
            index = 1

        # rotate first
        a = np.cross(A[index], (0,0,1))
        b = np.cross(B[index], (0, 0, 1))
        rot = scs.transform.Rotation.match_vectors([A[index], a], [B[index], b])[0]
        transformed = rot.apply(c.coordinates)
        #print (np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))

        # rotate second
        A = c.coordinates[[refsite1, refsite2]]
        B = transformed[[ontosite1, ontosite2]]
        rot = scs.transform.Rotation.match_vectors(A, B)[0]
        transformed2 = rot.apply(transformed)
        transformed = transformed2
        #print (np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))
        if max(np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1)) > 1e-5:
            raise ValueError('no rotation')
    
    if mirrorafter:
        a = transformed[ontosite1] - transformed[ontosite2]
        for site in range(20):
            v = transformed[site].copy()
            transformed[site] = v- 2*a*np.dot(v, a) / np.dot(a, a)
            
        a = transformed[ontosite1] + transformed[ontosite2]
        a = a/np.linalg.norm(a)*np.pi
        rot = scs.transform.Rotation.from_rotvec(a)
        transformed = rot.apply(transformed)
        #print (np.linalg.norm(c.coordinates[[refsite1, refsite2]] - transformed[[ontosite1, ontosite2]] , axis=1))
        
    found = []
    for site in range(len(c.coordinates)):
        ds = np.linalg.norm(transformed - c.coordinates[site], axis=1)
        if min(ds) < 1e-5:
            found.append(np.argmin(ds))
    #print (set(found))
    if set(found) == set([_ for _ in range(len(c.coordinates))]) and found[refsite1] == ontosite1 and found[refsite2] == ontosite2:
        pass
    else:
        raise ValueError('no solution')
    return found

In [320]:
build_reindexing_2_merged(1, 6, 0, 2)

[4, 0, 5, 14, 13, 3, 2, 1, 7, 6, 19, 18, 17, 12, 11, 8, 15, 16, 10, 9]

In [247]:
c.coordinates[[1, 2]].sum(axis=0)

array([-1.20877346,  1.66373394,  3.32746666])